In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR100
from torch.utils.data import DataLoader
from src.inception_cifar100 import inceptionv3

train_transforms = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762))
])

test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762))
])

train_dataset = CIFAR100(root="../../data", train=True, transform=train_transforms, download=True)
test_dataset = CIFAR100(root="../../data", train=False, transform=test_transforms, download=True)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = inceptionv3()
model = model.to(device)

def evaluate_model(model, test_loader, device):
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy  

def train_model(model, train_loader, criterion, optimizer, device, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss, correct, total = 0.0, 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, Accuracy: {accuracy:.2f}%\n")
        
        # Test accuracy when train-accuracy is above 90%
        if accuracy >= 90:
            print("Testing Accuracy on Test Set...")
            test_accuracy = evaluate_model(model, test_loader, device)
            if test_accuracy >= 91:
                print(f"Model accuracy reaches {test_accuracy}% at epoch {epoch}")
                print("Training Finished!")
                break
            else:
                print("Model haven't arrived 91% accuracy. Training continue!\n")
                model.train()

Files already downloaded and verified
Files already downloaded and verified


In [2]:
criterion = nn.CrossEntropyLoss()
optimizer_3 = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
optimizer_4 = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)


train_model(model, train_loader, criterion, optimizer_3, device, num_epochs=20)
train_model(model, train_loader, criterion, optimizer_4, device, num_epochs=5)

Epoch 1/20, Loss: 4.2263, Accuracy: 4.46%

Epoch 2/20, Loss: 3.8515, Accuracy: 9.41%

Epoch 3/20, Loss: 3.5363, Accuracy: 14.92%

Epoch 4/20, Loss: 3.2498, Accuracy: 19.75%

Epoch 5/20, Loss: 3.0018, Accuracy: 24.54%

Epoch 6/20, Loss: 2.7929, Accuracy: 28.75%

Epoch 7/20, Loss: 2.6042, Accuracy: 32.66%

Epoch 8/20, Loss: 2.4465, Accuracy: 35.96%

Epoch 9/20, Loss: 2.3024, Accuracy: 39.06%

Epoch 10/20, Loss: 2.1878, Accuracy: 41.62%

Epoch 11/20, Loss: 2.0907, Accuracy: 43.85%

Epoch 12/20, Loss: 2.0110, Accuracy: 45.88%

Epoch 13/20, Loss: 1.9402, Accuracy: 47.37%

Epoch 14/20, Loss: 1.8690, Accuracy: 48.97%

Epoch 15/20, Loss: 1.8051, Accuracy: 50.38%

Epoch 16/20, Loss: 1.7660, Accuracy: 51.62%

Epoch 17/20, Loss: 1.7096, Accuracy: 52.99%

Epoch 18/20, Loss: 1.6757, Accuracy: 53.81%

Epoch 19/20, Loss: 1.6453, Accuracy: 54.62%

Epoch 20/20, Loss: 1.6067, Accuracy: 55.21%

Epoch 1/5, Loss: 1.4542, Accuracy: 59.18%

Epoch 2/5, Loss: 1.3777, Accuracy: 61.19%

Epoch 3/5, Loss: 1.3493, 

In [3]:
train_transforms = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762))
])
train_dataset = CIFAR100(root="../../data", train=True, transform=train_transforms, download=True)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)

Files already downloaded and verified


In [6]:
train_model(model, train_loader, criterion, optimizer_3, device, num_epochs=5)

Epoch 1/5, Loss: 1.4501, Accuracy: 58.88%

Epoch 2/5, Loss: 1.4153, Accuracy: 59.89%

Epoch 3/5, Loss: 1.3885, Accuracy: 60.74%

Epoch 4/5, Loss: 1.3528, Accuracy: 61.39%

Epoch 5/5, Loss: 1.3215, Accuracy: 62.30%



In [7]:
train_model(model, train_loader, criterion, optimizer_3, device, num_epochs=10)

Epoch 1/10, Loss: 1.2968, Accuracy: 62.74%

Epoch 2/10, Loss: 1.2729, Accuracy: 63.46%

Epoch 3/10, Loss: 1.2547, Accuracy: 63.90%

Epoch 4/10, Loss: 1.2271, Accuracy: 64.70%

Epoch 5/10, Loss: 1.2107, Accuracy: 64.96%

Epoch 6/10, Loss: 1.1921, Accuracy: 65.75%

Epoch 7/10, Loss: 1.1702, Accuracy: 66.18%

Epoch 8/10, Loss: 1.1572, Accuracy: 66.39%

Epoch 9/10, Loss: 1.1416, Accuracy: 66.68%

Epoch 10/10, Loss: 1.1226, Accuracy: 67.42%



In [8]:
optimizer_4 = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=5e-5)
train_model(model, train_loader, criterion, optimizer_4, device, num_epochs=5)

Epoch 1/5, Loss: 0.8716, Accuracy: 74.40%

Epoch 2/5, Loss: 0.7936, Accuracy: 76.45%

Epoch 3/5, Loss: 0.7576, Accuracy: 77.28%

Epoch 4/5, Loss: 0.7203, Accuracy: 78.31%

Epoch 5/5, Loss: 0.7030, Accuracy: 78.91%



In [9]:
train_model(model, train_loader, criterion, optimizer_4, device, num_epochs=5)

Epoch 1/5, Loss: 0.6804, Accuracy: 79.41%

Epoch 2/5, Loss: 0.6660, Accuracy: 79.74%

Epoch 3/5, Loss: 0.6497, Accuracy: 80.38%

Epoch 4/5, Loss: 0.6316, Accuracy: 80.66%

Epoch 5/5, Loss: 0.6183, Accuracy: 80.95%



In [10]:
test_accuracy = evaluate_model(model, test_loader, device)
print(f"Current Accuracy on Test Set: {test_accuracy}%")

Current Accuracy on Test Set: 70.08%


In [11]:
optimizer_5 = torch.optim.Adam(model.parameters(), lr=5e-5, weight_decay=1e-5)
train_model(model, train_loader, criterion, optimizer_5, device, num_epochs=5)

Epoch 1/5, Loss: 0.5976, Accuracy: 81.78%

Epoch 2/5, Loss: 0.5879, Accuracy: 81.99%

Epoch 3/5, Loss: 0.5743, Accuracy: 82.40%

Epoch 4/5, Loss: 0.5731, Accuracy: 82.61%

Epoch 5/5, Loss: 0.5592, Accuracy: 82.81%



In [16]:
train_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762))
])
train_dataset = CIFAR100(root="../../data", train=True, transform=train_transforms, download=True)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)

train_model(model, train_loader, criterion, optimizer_5, device, num_epochs=5)

Files already downloaded and verified
Epoch 1/5, Loss: 0.5017, Accuracy: 84.39%

Epoch 2/5, Loss: 0.4442, Accuracy: 86.38%

Epoch 3/5, Loss: 0.4268, Accuracy: 86.71%

Epoch 4/5, Loss: 0.4066, Accuracy: 87.41%

Epoch 5/5, Loss: 0.3850, Accuracy: 87.99%



In [17]:
train_model(model, train_loader, criterion, optimizer_5, device, num_epochs=5)

Epoch 1/5, Loss: 0.3730, Accuracy: 88.35%

Epoch 2/5, Loss: 0.3561, Accuracy: 88.98%

Epoch 3/5, Loss: 0.3437, Accuracy: 89.31%

Epoch 4/5, Loss: 0.3259, Accuracy: 89.99%

Epoch 5/5, Loss: 0.3143, Accuracy: 90.25%

Testing Accuracy on Test Set...
Model haven't arrived 91% accuracy. Training continue!



In [20]:
import os
save_dir = "C:\\Users\\jiayang\\ipynb\\trainedModel" # your save save path
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
model_path = os.path.join(save_dir, "Inception_CIFAR10.pth")
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

Model saved to C:\Users\jiayang\ipynb\trainedModel\Inception_CIFAR10.pth
